# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [29]:
import pandas as pd
import requests

url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
response = requests.get(url)
data = response.json()



In [31]:
# Convertendo os dados em um DataFrame
df = pd.DataFrame(data['items'])

In [34]:
# Verificando as colunas
print(df.columns)

Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')


In [40]:
# Passo 2: Filtrar os dados para incluir apenas os PVLs realizados por municípios no estado do Rio de Janeiro
df_rj_municipios = df[(df['tipo_interessado'] == 'Município') & (df['uf'] == 'RJ')]


In [42]:
# Passo 3: Encontrar os três status das solicitações mais frequentes e suas frequências
status_frequencia = df_rj_municipios['status'].value_counts().head(3)
print("Os três status das solicitações mais frequentes e suas frequências:")
print(status_frequencia)

Os três status das solicitações mais frequentes e suas frequências:
status
Deferido                          28
Arquivado                         16
Arquivado por decurso de prazo     9
Name: count, dtype: int64


In [48]:
# Passo 4: Construir uma nova variável que contenha o ano do status
df_rj_municipios.loc[:, 'ano_status'] = df_rj_municipios['data_status'].str.slice(0, 4)

In [50]:
# Passo 5: Indicar a frequência de cada ano do campo construído
ano_frequencia = df_rj_municipios['ano_status'].value_counts()
print("\nFrequência de cada ano do campo 'ano_status':")
print(ano_frequencia)


Frequência de cada ano do campo 'ano_status':
ano_status
15/0    5
29/0    5
13/0    3
01/1    3
10/0    3
22/0    3
26/0    3
25/0    2
27/0    2
06/0    2
01/0    2
23/0    2
18/0    2
19/1    2
30/1    2
30/0    2
31/0    2
22/1    1
08/1    1
29/1    1
12/0    1
13/1    1
23/1    1
11/1    1
04/1    1
17/1    1
09/0    1
02/0    1
21/1    1
21/0    1
04/0    1
18/1    1
31/1    1
11/0    1
10/1    1
20/0    1
17/0    1
03/0    1
07/0    1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [52]:
import pandas as pd
import requests

def consulta_pvl(uf, tipo_interessado):
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['items'])
    
    # Filtrar os dados com base nos parâmetros fornecidos
    df_filtrado = df[(df['uf'] == uf) & (df['tipo_interessado'] == tipo_interessado)]
    
    return df_filtrado


In [54]:
# Testando a função com o UF 'RJ' e tipo de interessado 'Município'
df_rj_municipios = consulta_pvl('RJ', 'Município')
print(df_rj_municipios.head())

     id_pleito tipo_interessado          interessado  cod_ibge  uf  \
110      10108        Município  São Pedro da Aldeia   3305208  RJ   
190       8590        Município         Belford Roxo   3300456  RJ   
250      66729        Município       Miguel Pereira   3302908  RJ   
293      65489        Município          Nova Iguaçu   3303500  RJ   
418      13638        Município          São Gonçalo   3304904  RJ   

                  num_pvl                          status  \
110                  None                       Arquivado   
190                  None  Arquivado por decurso de prazo   
250  PVL02.003701/2023-85               Deferido (PVL-IF)   
293  PVL02.002202/2023-71                        Deferido   
418                  None                       Arquivado   

             num_processo        data_protocolo                tipo_operacao  \
110  17944.001122/2008-21  2009-03-06T03:00:00Z  Operação contratual interna   
190  17944.000804/2011-12  2014-10-20T02:00:00Z  Ope

In [56]:
#1) Consultar Solicitações para Minas Gerais com Status "Arquivado por Decurso de Prazo
df_mg_arquivado = consulta_pvl('MG', 'Estado')
df_mg_arquivado_prazo = df_mg_arquivado[df_mg_arquivado['status'] == 'Arquivado por decurso de prazo']
quantidade_arquivado_prazo = len(df_mg_arquivado_prazo)
print(f"Quantidade de solicitações para Minas Gerais com status 'Arquivado por decurso de prazo': {quantidade_arquivado_prazo}")


Quantidade de solicitações para Minas Gerais com status 'Arquivado por decurso de prazo': 0


In [58]:
# 2)  Identificar o Município da Bahia com Mais Solicitações Deferidas
df_ba_municipios = consulta_pvl('BA', 'Município')
df_ba_deferidas = df_ba_municipios[df_ba_municipios['status'] == 'Deferido']
municipio_com_mais_deferidas = df_ba_deferidas['interessado'].value_counts().idxmax()
quantidade_mais_deferidas = df_ba_deferidas['interessado'].value_counts().max()
print(f"Município da Bahia com mais solicitações deferidas: {municipio_com_mais_deferidas} ({quantidade_mais_deferidas} solicitações)")


Município da Bahia com mais solicitações deferidas: Camaçari (3 solicitações)


In [60]:
# 3) Salvar um Arquivo .csv com os Dados de Solicitações da Bahia com Interessado = "Estado"
df_ba_estado = consulta_pvl('BA', 'Estado')
df_ba_estado.to_csv('solicitacoes_ba_estado.csv', index=False)
print("Arquivo 'solicitacoes_ba_estado.csv' salvo com sucesso.")


Arquivo 'solicitacoes_ba_estado.csv' salvo com sucesso.


In [ ]:
# 4) Seu código aqui